In [ ]:
!pip install peft datasets transformers trl accelerate bitsandbytes -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 12.5 MB/s eta 0:00:00


In [ ]:
import torch
import json
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, GenerationConfig
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from peft import LoraConfig, PeftModelForCausalLM
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from datasets import Dataset
from huggingface_hub import notebook_login
from google.colab import userdata

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
save_path = "/content/drive/MyDrive/Shared/TheBucket/Datasets/OASST2/Processed/"
sa_ds_filepath = "/content/drive/MyDrive/Shared/TheBucket/Datasets/StanceClassification/ID/testFile1Simple_BeritaFull_cmb.csv"

dataset_train_name = 'ID-train-EN_GT'
dataset_val_name = 'ID-val-EN_GT'

file_name_train_chatml = f"{dataset_train_name}_chatml.json"
file_name_val_chatml = f"{dataset_val_name}_chatml.json"

file_name_train = f"{dataset_train_name}.json"
file_name_val = f"{dataset_val_name}.json"


In [ ]:
with open(save_path + file_name_train_chatml, 'r') as f:
  json_train = json.load(f)
  print(json_train['messages'][0])

with open(save_path + file_name_val_chatml, 'r') as f:
  json_val = json.load(f)
  print(json_val['messages'][0])

[{'role': 'user', 'content': 'Saya membuat mayones, itu mulai menebal tetapi sekarang telah menjadi berair dan cair lagi, apakah ada cara untuk menyelamatkannya?'}, {'role': 'assistant', 'content': 'Ya, mungkin untuk memperbaiki mayones berair!Alasan paling umum untuk mayones menjadi berair adalah karena minyak ditambahkan terlalu cepat atau kuning telur tidak diemulsi dengan benar.Berikut adalah beberapa langkah yang dapat Anda ambil untuk memperbaikinya:\n\n1. Pisahkan kuning telur lain dan letakkan di dalam mangkuk yang bersih dan kering.\n2. Perlahan tambahkan mayones yang berair ke kuning telur sambil mengocok dengan kuat.\n3. Setelah semua mayones orny ditambahkan, terus mengocok sampai campuran diemulsi dan mengental.\n4. Jika mayones masih terlalu berair, Anda dapat menambahkan kuning telur lain dan mengulangi prosesnya.\n\nJika mayones masih tidak akan menebal, Anda dapat mencoba menambahkan sedikit mustard atau cuka ke dalam campuran, yang dapat bertindak sebagai pengemulsi d

In [ ]:
dataset_train = Dataset.from_dict(json_train)
dataset_val = Dataset.from_dict(json_val)

In [ ]:
bnb_config = BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_quant_type="nf4",
  bnb_4bit_compute_dtype=torch.float16,
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it", token=userdata.get("HFReadToken"))
tokenizer.padding_side = 'left'
tokenizer.truncation_side = "left"
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2b-it",
    device_map="auto",
    quantization_config=bnb_config,
    token=userdata.get("HFReadToken")
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to s

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
original_chat_template = tokenizer.chat_template
chat_template_for_training = """{{ bos_token }}{% if messages[0]['role'] == 'system' %}{{ raise_exception('System role not supported') }}{% endif %}{% for message in messages %}{% if (message['role'] == 'user') != (loop.index0 % 2 == 0) %}{{ raise_exception('Conversation roles must alternate user/assistant/user/assistant/...') }}{% endif %}{% if (message['role'] == 'assistant') %}{% set role = 'model' %}{% else %}{% set role = message['role'] %}{% endif %}{{ '<start_of_turn>' + role + '
' + message['content'] | trim }}{% if loop.last and not add_generation_prompt and message['role'] == 'assistant'%}{{ '<eos>' }}{% endif %}{{ '<end_of_turn>
' }}{% endfor %}{% if add_generation_prompt %}{{'<start_of_turn>model
'}}{% endif %}"""
tokenizer.chat_template = chat_template_for_training

In [ ]:
print(model)

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 3072, padding_idx=0)
    (layers): ModuleList(
      (0-27): 28 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear4bit(in_features=3072, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=3072, out_features=4096, bias=False)
          (v_proj): Linear4bit(in_features=3072, out_features=4096, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=3072, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear4bit(in_features=3072, out_features=24576, bias=False)
          (up_proj): Linear4bit(in_features=3072, out_features=24576, bias=False)
          (down_proj): Linear4bit(in_features=24576, out_features=3072, bias=False)
          (act_fn): GELUActivation()
        )
        (input_layernorm): GemmaRMSNorm()
        (post_attention_layernorm): GemmaRMSNorm()
   

In [ ]:
output_dir = "/content/drive/MyDrive/Shared/TheBucket/Files/GemmaFinetuneID/oasst2-id-e1-2B_1"
# output_dir = "/content/drive/MyDrive/Files/GemmaFinetuneID/oasst2-id-e1-7B_2"
# output_dir = "/content/drive/MyDrive/AsDos/GemmaFinetuneID/oasst2-id-e1-7B_2"

In [ ]:
peft_config = LoraConfig(
    lora_alpha = 16,
    lora_dropout=0.1,
    r=16,
    task_type='CAUSAL_LM'
)


In [ ]:


training_arguments = TrainingArguments(
    output_dir = output_dir,
    evaluation_strategy="steps",
    logging_strategy="steps",
    lr_scheduler_type="constant",
    logging_steps=15,
    eval_steps=256,
    save_steps=15,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=50,
    eval_accumulation_steps=50,
    num_train_epochs=1,
    fp16=True,
    group_by_length = True,
    optim="paged_adamw_32bit",
    # max_steps = 180
)

trainer = SFTTrainer(
    model,
    tokenizer=tokenizer,
    train_dataset=dataset_train,
    eval_dataset=dataset_val,
    peft_config=peft_config,
    neftune_noise_alpha=5,
    max_seq_length=256,
    args = training_arguments,

)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:257: UserWarning: You passed a `neftune_noise_alpha` argument to the SFTTrainer, the value you passed will override the one in the `TrainingArguments`.
  warnings.warn(


Map:   0%|          | 0/38439 [00:00<?, ? examples/s]

Map:   0%|          | 0/2047 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [ ]:
trainer.train()

In [ ]:
trainer.train(resume_from_checkpoint=True)

Step,Training Loss,Validation Loss


/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:581: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-6638cb16-3170d71751ee77d2619e37e7;3dcdc381-c3a2-4c41-875d-fd3905b56665)

Cannot access gated repo for url https://huggingface.co/google/gemma-2b-it/resolve/main/config.json.
Access to model google/gemma-2b-it is restricted. You must be authenticated to access it. - silently ignoring the lookup for the file config.json in google/gemma-2b-it.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in google/gemma-2b-it - will assume that the vocabulary was not modified.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:581: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-6638cd1b-5da33a637db6c30b13bb6bcb;008ed2ba-e89e-4f29-b8d0-d68082661dff)

Cannot access gated

TrainOutput(global_step=192, training_loss=1.4486126104990642, metrics={'train_runtime': 2945.0096, 'train_samples_per_second': 13.052, 'train_steps_per_second': 0.065, 'total_flos': 9.523325601629798e+16, 'train_loss': 1.4486126104990642, 'epoch': 0.9989594172736732})

In [ ]:
# save adapters
trainer.model.save_pretrained("/content/drive/MyDrive/Shared/TheBucket/Files/GemmaFinetuneID/oasst2-id-e1-2B_1/adapters/01")

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:581: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-6638d41a-2bb6a5fa4467649d49c47feb;89fc731e-1fda-4147-9baf-ebe5d004ac68)

Cannot access gated repo for url https://huggingface.co/google/gemma-2b-it/resolve/main/config.json.
Access to model google/gemma-2b-it is restricted. You must be authenticated to access it. - silently ignoring the lookup for the file config.json in google/gemma-2b-it.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in google/gemma-2b-it - will assume that the vocabulary was not modified.
  warnings.warn(


In [ ]:
finetuned_model = PeftModelForCausalLM.from_pretrained(model=model, model_id="/content/drive/MyDrive/Shared/TheBucket/Files/GemmaFinetuneID/oasst2-id-e1-2B_1/adapters/01")
# finetuned_model = PeftModelForCausalLM.from_pretrained(model=model, model_id="/content/drive/MyDrive/AsDos/GemmaFinetune/adapters/testFileSimple_sa/epoch_3")
# load_model = PeftModelForCausalLM.from_pretrained(model=model ,model_id="/content/drive/MyDrive/AsDos/GemmaFinetune/02/checkpoint-5")

In [ ]:
istts_ds_filepath = "buku_pedoman_istts_clean.csv"
RANDOM_STATE = 420
istts_df = pd.read_csv(istts_ds_filepath, index_col=0)
istts_df = istts_df.dropna()


In [ ]:
istts_df

,text
0,"REKTOR\nIr. Arya Tandy Hermawan, M.T.\nPenyele..."
1,TRI DHARMA PENDIDIKAN\n• Dharma Pendidikan dan...
2,LAMBANG\nLambang ISTTS mempunyai arti sebagai ...
3,SEJARAH\n1979\nISTTS pada mulanya dikenal deng...
4,TUJUAN PENDIDIKAN\nSebagaimana telah dicanangk...
...,...
134,DESKRIPSI S2 TEKNOLOGI INFORMASI\nProgram Stud...
135,FOKUS PENDIDIKAN S2 TEKNOLOGI INFORMASI\nDalam...
136,KOMPETENSI LULUSAN DAN PROSPEK KARIR S2 TEKNOL...
137,MATA KULIAH PILIHAN SPESIALISASI\nArtificial I...


In [ ]:
istts_df_train = istts_df
istts_df_val = istts_df.iloc[-14:]

In [ ]:
ds_train_istts = Dataset.from_pandas(istts_df_train)
ds_val_istts = Dataset.from_pandas(istts_df_val)

In [ ]:
ds_val_istts

Dataset({
    features: ['text', '__index_level_0__'],
    num_rows: 12
})

In [ ]:
istts_output_dir = "/content/drive/MyDrive/Shared/TheBucket/Files/GemmaFinetuneID/ElangBot"

In [ ]:
merged_finetuned_model = finetuned_model.merge_and_unload()

/usr/local/lib/python3.10/dist-packages/peft/tuners/lora/bnb.py:325: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [ ]:
max([len(tokenizer.encode(row)) for row in istts_df['text']])


1463

In [ ]:
training_arguments_istts = TrainingArguments(
    output_dir = istts_output_dir,
    evaluation_strategy="steps",
    logging_strategy="steps",
    lr_scheduler_type="constant",
    logging_steps=5,
    eval_steps=5,
    save_steps=5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=16,
    eval_accumulation_steps=16,
    num_train_epochs=51,
    fp16=True,
    group_by_length = True,
    optim="paged_adamw_32bit",
    # max_steps = 180
)

def formatting_prompts_func(data):
    return data['text']

trainer_istts = SFTTrainer(
    merged_finetuned_model,
    tokenizer=tokenizer,
    peft_config=peft_config,
    train_dataset=ds_train_istts,
    eval_dataset=ds_val_istts,
    neftune_noise_alpha=5,
    max_seq_length=1024,
    args = training_arguments_istts,
    formatting_func=formatting_prompts_func,
    packing=True
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:257: UserWarning: You passed a `neftune_noise_alpha` argument to the SFTTrainer, the value you passed will override the one in the `TrainingArguments`.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [ ]:
trainer_istts.train()

Step,Training Loss,Validation Loss
5,4.443000,4.705535
10,4.439200,4.610505
15,4.188000,4.490165
20,4.150000,4.347081
25,3.966900,4.207631
30,3.820400,4.081335
35,3.739000,3.970623
40,3.643200,3.879400
45,3.550000,3.804846
50,3.501400,3.742272


/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:581: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-66412949-417604a17c03d7f03c80c756;cc28c5aa-abf7-441a-9d49-d4d9f94f07f8)

Cannot access gated repo for url https://huggingface.co/google/gemma-2b-it/resolve/main/config.json.
Access to model google/gemma-2b-it is restricted. You must be authenticated to access it. - silently ignoring the lookup for the file config.json in google/gemma-2b-it.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in google/gemma-2b-it - will assume that the vocabulary was not modified.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:581: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-664129a1-3a89c10a02f37fb45f073eef;44bf06ce-250f-4c7d-b5af-ae601c39a812)

Cannot access gated

TrainOutput(global_step=51, training_loss=3.9358511812546673, metrics={'train_runtime': 873.1337, 'train_samples_per_second': 1.694, 'train_steps_per_second': 0.058, 'total_flos': 9945426337726464.0, 'train_loss': 3.9358511812546673, 'epoch': 28.137931034482758})

In [ ]:
trainer_istts.train(resume_from_checkpoint=True)

In [ ]:
trainer_istts.model.save_pretrained(istts_output_dir + "/man_save_e51")

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:581: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-66412c63-6b5175433776c7131d6317bf;574dd5bf-c155-42d5-b25a-cfad18d475c5)

Cannot access gated repo for url https://huggingface.co/google/gemma-2b-it/resolve/main/config.json.
Access to model google/gemma-2b-it is restricted. You must be authenticated to access it. - silently ignoring the lookup for the file config.json in google/gemma-2b-it.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in google/gemma-2b-it - will assume that the vocabulary was not modified.
  warnings.warn(


In [ ]:
finetuned_istts_model = PeftModelForCausalLM.from_pretrained(model=merged_finetuned_model, model_id=istts_output_dir + "/man_save_e51")

In [ ]:
merged_finetuned_istts_model = finetuned_istts_model.merge_and_unload()

In [ ]:
istts_it_output_dir = "/content/drive/MyDrive/Shared/TheBucket/Files/GemmaFinetuneID/ElangBot-it"

In [ ]:
original_chat_template = tokenizer.chat_template
chat_template_for_training = """{{ bos_token }}{% if messages[0]['role'] == 'system' %}{{ raise_exception('System role not supported') }}{% endif %}{% for message in messages %}{% if (message['role'] == 'user') != (loop.index0 % 2 == 0) %}{{ raise_exception('Conversation roles must alternate user/assistant/user/assistant/...') }}{% endif %}{% if (message['role'] == 'assistant') %}{% set role = 'model' %}{% else %}{% set role = message['role'] %}{% endif %}{{ '<start_of_turn>' + role + '
' + message['content'] | trim }}{% if loop.last and not add_generation_prompt and message['role'] == 'assistant'%}{{ '<eos>' }}{% endif %}{{ '<end_of_turn>
' }}{% endfor %}{% if add_generation_prompt %}{{'<start_of_turn>model
'}}{% endif %}"""
tokenizer.chat_template = chat_template_for_training

In [ ]:
istts_json_train_filepath = "gemini_generated_chatml.json"
with open(istts_json_train_filepath, 'r') as f:
  istts_json_train = json.load(f)
  print(istts_json_train['messages'][0])
istts_dataset_train = Dataset.from_dict(istts_json_train)

[{'role': 'user', 'content': 'Apa misi ISTTS?'}, {'role': 'assistant', 'content': 'ISTTS memiliki 3 misi utama:\n1. Memberikan pendidikan dan pengajaran terbaik di bidang sains, teknologi, dan desain, dengan sistem tata kelola yang efisien dan sesuai kebutuhan masyarakat Indonesia.\n2. Bekerja sama dengan perguruan tinggi lain di dalam dan luar negeri untuk melakukan penelitian dan pengembangan di bidang sains, teknologi, dan desain, yang sesuai dengan kondisi dan kebutuhan masyarakat Indonesia.\n3. Menyediakan solusi inovatif dan unggul berbasis sains, teknologi, dan desain untuk berbagai masalah sosial, ekonomi, dan budaya, khususnya untuk membantu pemerintah Indonesia mewujudkan superioritas kekuatan sosial, ekonomi, dan budaya Indonesia.'}]


In [ ]:
peft_config = LoraConfig(
    lora_alpha = 32,
    lora_dropout=0.1,
    r=16,
    task_type='CAUSAL_LM'
)


istts_it_training_arguments = TrainingArguments(
    output_dir = istts_it_output_dir,
    # evaluation_strategy="steps",
    logging_strategy="steps",
    lr_scheduler_type="constant",
    logging_steps=5,
    # eval_steps=10,
    save_steps=10,
    per_device_train_batch_size=4,
    # per_device_eval_batch_size=4,
    gradient_accumulation_steps=32,
    # eval_accumulation_steps=32,
    num_train_epochs=400,
    fp16=True,
    group_by_length = True,
    optim="paged_adamw_32bit",
    # max_steps = 180
)

istts_it_trainer = SFTTrainer(
    merged_finetuned_istts_model,
    tokenizer=tokenizer,
    train_dataset=istts_dataset_train,
    # eval_dataset=dataset_val,
    peft_config=peft_config,
    neftune_noise_alpha=5,
    max_seq_length=256,
    args = istts_it_training_arguments,

)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:257: UserWarning: You passed a `neftune_noise_alpha` argument to the SFTTrainer, the value you passed will override the one in the `TrainingArguments`.
  warnings.warn(


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [ ]:
istts_it_trainer.train()

Step,Training Loss
5,3.517400
10,3.367600
15,3.109600
20,2.451000
25,2.351000
30,2.459600
35,2.308500
40,2.145400
45,1.716000
50,1.634600


/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:581: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-6641328a-2472f7513102f5ce299954d3;2c192e27-5a3a-4007-999b-c70ff216970b)

Cannot access gated repo for url https://huggingface.co/google/gemma-2b-it/resolve/main/config.json.
Access to model google/gemma-2b-it is restricted. You must be authenticated to access it. - silently ignoring the lookup for the file config.json in google/gemma-2b-it.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in google/gemma-2b-it - will assume that the vocabulary was not modified.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:581: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-664132c5-01e5b9931b799d274cfbb926;e4e69f8d-2e96-4dff-bb0e-63b9bb57b8cf)

Cannot access gated

OutOfMemoryError: CUDA out of memory. Tried to allocate 840.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 369.06 MiB is free. Process 1884 has 14.38 GiB memory in use. Of the allocated memory 13.41 GiB is allocated by PyTorch, and 858.93 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
istts_it_trainer.train(resume_from_checkpoint=True)

Step,Training Loss
305,0.419100
310,0.410800
315,0.398500
320,0.335500
325,0.339300
330,0.374800
335,0.377500
340,0.370100
345,0.310100
350,0.307300


/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:581: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-6641416b-5de34385651bda4e6d95ce71;2fe2291f-122e-4a0f-9c8c-f3cf89e77671)

Cannot access gated repo for url https://huggingface.co/google/gemma-2b-it/resolve/main/config.json.
Access to model google/gemma-2b-it is restricted. You must be authenticated to access it. - silently ignoring the lookup for the file config.json in google/gemma-2b-it.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in google/gemma-2b-it - will assume that the vocabulary was not modified.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:581: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-664141a7-64f1417c54d6b91c44011d60;08d5bf26-6240-48ea-b6d9-5f6690eed862)

Cannot access gated

TrainOutput(global_step=400, training_loss=0.08566457003355027, metrics={'train_runtime': 623.3019, 'train_samples_per_second': 64.174, 'train_steps_per_second': 0.642, 'total_flos': 2.0266205680041984e+16, 'train_loss': 0.08566457003355027, 'epoch': 357.0})

In [ ]:
istts_it_trainer.model.save_pretrained(istts_it_output_dir + "/man_save_la32_e300")

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:581: UserWarning: Unable to fetch remote file due to the following error 401 Client Error. (Request ID: Root=1-66413cc6-6b7aaa6014b1e1e43d066358;0a3943e4-6a5f-4a45-b4c0-4318ae975171)

Cannot access gated repo for url https://huggingface.co/google/gemma-2b-it/resolve/main/config.json.
Access to model google/gemma-2b-it is restricted. You must be authenticated to access it. - silently ignoring the lookup for the file config.json in google/gemma-2b-it.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in google/gemma-2b-it - will assume that the vocabulary was not modified.
  warnings.warn(


In [ ]:
finetuned_it_model = PeftModelForCausalLM.from_pretrained(model=merged_finetuned_istts_model, model_id=istts_it_output_dir + "/checkpoint-300")

## Chat Testing

In [ ]:
model_2 = AutoModelForCausalLM.from_pretrained(
    "google/gemma-7b-it",
    # device_map="auto",
    quantization_config=bnb_config
)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
def generate(model, tokenizer, history):
  return tokenizer.decode(
        model.generate(
              input_ids=tokenizer.apply_chat_template(
                    history,
                    add_generation_prompt=True,
                    tokenize=True,
                    return_tensors="pt"
                  ).to("cuda"),
              max_new_tokens=256,

              do_sample=False
            )[0]
      ).split('<start_of_turn>model\n')[-1]

In [ ]:
    # {
    #     'role':'assistant',
    #     'content':'Burung itu bisa terbang.',
    # },
    # {
    #     'role':'user',
    #     'content':'karena?',
    # }
messages=[
    {
        'role':'user',
        'content':'S1 informatika itu blajar apa?',
    },
]


generate(finetuned_it_model, tokenizer, messages)

'S1 Informatika merupakan pemrograman dasar Java. Students akan diPreparazionekan di Java Koding, Java Apa, dan Java Apa Apa.<eos>'

In [ ]:
generate(model, tokenizer, messages)

'Mangkuk adalah wadah yang biasanya terbuat dari bahan yang lebih ringan dan lebih mudah dihilangkan, seperti tanah, tanah, atau perunggu.Bersih-bersih biasanya terbuat dari bahan yang lebih berat dan lebih sulit dihilangkan, seperti perunggu, tembaga, atau emas.\n\nMangkuk biasanya digunakan untuk menyimpan air, makanan, atau minuman, sedangkan bersih-bersih biasanya digunakan untuk menyimpan bahan-bahan seperti bahan-bahan dasar, garam, atau gula.\n\nMangkuk biasanya memiliki bentuk yang lebih sederhana dan lebih kecil, sedangkan bersih-bersih biasanya memiliki bentuk yang lebih kompleks dan lebih besar.\n\nMangkuk biasanya memiliki lidah atau lidah yang lebih kecil, sedangkan bersih-bersih biasanya memiliki lidah atau lidah yang lebih besar.\n\nMangkuk biasanya memiliki lidah yang lebih kecil dan lebih pendek, sedangkan bersih-bersih biasanya memiliki lidah yang lebih besar dan lebih panjang.\n\nMangkuk biasanya memiliki lidah yang lebih kecil dan lebih pendek, sedangkan bersih-bers